Build a regression model.

Provide model output and an interpretation of the results. 

In [ ]:
import requests
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

url = "http://api.citybik.es/v2/networks"
response = requests.get(url)

if response.status_code == 200:
    response_data = response.json()
    networks = response_data.get('networks', [])
    vancouver_network = None
    
    for i in networks:
        if i['location']['city'].lower() == 'vancouver':
            vancouver_network = i
            break
    
    if vancouver_network:
        network_id = vancouver_network['id']
        details_url = f"http://api.citybik.es/v2/networks/{network_id}"
        details_response = requests.get(details_url)
        
        if details_response.status_code == 200:
            vancouver_details = details_response.json()
            stations = vancouver_details['network']['stations']
            
            data = [{'Name': station['name'], 'Latitude': station['latitude'], 'Longitude': station['longitude']} for station in stations]
            df1 = pd.DataFrame(data)
        else:
            raise Exception(f"Failed to retrieve details for Vancouver network: {details_response.status_code}")
    else:
        raise Exception("Vancouver network not found.")
else:
    raise Exception(f"Failed to retrieve data: {response.status_code}")

headers = {
    "Accept": "application/json",
    "Authorization": "fsq3XRUAarAkYkZaUxo7U14qSYIS91CYwbt1bF94/FymeqU="
}

url = "https://api.foursquare.com/v3/places/search"
params = {
    'radius': 100000,
    'll': '49.262487,-123.114397',
    'categories': '13065,13056', 
    'fields': 'name,geocodes'
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    response_data = response.json()
    places = response_data.get('results', [])
    
    dining_and_drinking = [
        {
            'Name': place.get('name'),
            'Latitude': place['geocodes']['main']['latitude'] if 'geocodes' in place and 'main' in place['geocodes'] else None,
            'Longitude': place['geocodes']['main']['longitude'] if 'geocodes' in place and 'main' in place['geocodes'] else None
        }
        for place in places
    ]
    
    df2 = pd.DataFrame(dining_and_drinking)
else:
    raise Exception(f"Error: {response.status_code}, {response.text}")

df1['Type'] = 'Bike Station'
df2['Type'] = 'Restaurant/Bar'
combined_df = pd.concat([df1, df2])

combined_df = combined_df.dropna(subset=['Latitude', 'Longitude'])
X = combined_df[['Longitude']]
y = combined_df['Latitude']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

plt.figure(figsize=(10, 6))
plt.scatter(combined_df['Longitude'], combined_df['Latitude'], c='blue', label='Data Points')
plt.plot(combined_df['Longitude'], model.predict(X), color='red', linewidth=2, label='Regression Line')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('OLS Regression Line for Latitude vs Longitude')
plt.legend()
plt.grid(True)
plt.show()

# Stretch

How can you turn the regression model into a classification model?

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import Binarizer

# Assuming df1 and df2 are already created and combined_df is prepared

# Prepare features and target variable
combined_df = combined_df.dropna(subset=['Latitude', 'Longitude'])
X = combined_df[['Longitude']]
y = combined_df['Latitude']

bins = [-float('inf'), 49.25, 49.30, float('inf')]
labels = ['Low', 'Medium', 'High']
y_binned = pd.cut(y, bins=bins, labels=labels)

scaler = Binarizer()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_binned, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))